# Chapter 3: Heat Diffusion in Two Dimensions

Building upon our success from the 1D models, it is now prudent to expand our domain by another dimension.
For this example we use a very simple magmatic intrusion as the basis for our model. The simulation will be
a single event where some molten granite has formed a cylindrical dome at the base of some cold sandstone
country rock. Assuming that the cylinder is very long we model a cross-section as shown in Figure 3.1. We
will implement the same diffusion model as we have used for the granite blocks in Section 2.1 but will add the
second spatial dimension and show how to define variables depending on the location of the domain. We use
`onedheatdiff001b.py` as the starting point to develop this model.

<br>
 <figure>
  <img src="figures/twodheatdiff.svg" width="400">
  <figcaption>
    <center>
      Figure 3.1: Example 3: 2D model: granitic intrusion of sandstone country rock.
    </center>
  </figcaption>
 </figure>
<br>

## 3.1 Example 3: Two Dimensional Heat Diffusion for a basic Magmatic Intrusion

**The scripts referenced in this section are: example03a.py and cblib.py**

To expand upon our 1D problem, the domain must first be expanded. In fact, we have solved a two dimensional problem already but essentially ignored the second dimension. In our definition phase we create a square domain in $x$ and $y$ (in *escript* the notation $x_{0}$ and $x_{1}$ is used for $x$ and $y$, respectively) that is 600 meters along each side (Figure 3.1). Now we set the number of discrete spatial cells to 150 in both directions and the radius of the intrusion to 200 meters with the centre located at the 300 meter mark on the $x$-axis. Thus, the domain variables are:

In [ ]:
from esys.escript import *
from esys.escript.unitsSI import *
from esys.finley import Rectangle

mx = 600*m #meters - model length
my = 600*m #meters - model width
ndx = 150 #mesh steps in x direction
ndy = 150 #mesh steps in y direction
r = 200*m #meters - radius of intrusion
ic = [300*m, 0] #coordinates of the centre of intrusion (meters)
qH=0.*J/(sec*m**3) #our heat source temperature is zero

model = Rectangle(l0=mx,l1=my,n0=ndx, n1=ndy)

print("Done")

As before, we created a domain using the `Rectangle()` function from *finley*. There are two fundamental changes to the PDE that we have discussed in Section 2.1. Firstly, because the material coefficients for granite and sandstone are different, we need to deal with PDE coefficients which vary with their location in the domain. Secondly, we need to deal with the second spatial dimension. We can investigate these two modifications at the same time. In fact, the temperature model Equation (2.1) we utilised in Section 2.1 applied for the 1D case with a constant material parameter only. For the more general case examined in this chapter, the correct model equation is:

\begin{equation}
  \rho c_p \frac{\partial T}{\partial t} 
  - \frac{\partial }{\partial x} \kappa \frac{\partial T}{\partial x} 
  - \frac{\partial }{\partial y} \kappa \frac{\partial T}{\partial y} = q_H .
  \tag{3.1}
\end{equation}

Notice that for the 1D case we have $\frac{\partial T}{\partial y}=0$ and for the case of constant material parameters $\frac{\partial }{\partial x} \kappa = \kappa  \frac{\partial }{\partial x}$ thus this new equation coincides with a simplified model equation for this case. It is more convenient to write this equation using the $\nabla$ notation as we have already seen in Equation (2.6):

\begin{equation}
  \rho c_p \frac{\partial T}{\partial t} - \nabla \cdot \kappa \nabla T = q_H .
  \tag{3.2}
\end{equation}

We can easily apply the backward Euler scheme as before to end up with 

\begin{equation}
  \frac{\rho c_p}{h} T^{(n)} -\nabla \cdot \kappa \nabla T^{(n)}  =
  q_H +  \frac{\rho c_p}{h} T^{(n-1)} ,
  \tag{3.3}
\end{equation}

which is very similar to Equation (2.9) used to define the temperature update in the simple 1D case. The difference is in the second order derivative term $\nabla \cdot \kappa \nabla T^{(n)}$. Under the light of the more general case we need to revisit the *escript* PDE form as shown in Equation (2.16). For the 2D case with variable PDE coefficients the form needs to be read as

\begin{equation}
  -\frac{\partial }{\partial x} A_{00}\frac{\partial u}{\partial x} 
  -\frac{\partial }{\partial x} A_{01}\frac{\partial u}{\partial y} 
  -\frac{\partial }{\partial y} A_{10}\frac{\partial u}{\partial x} 
  -\frac{\partial }{\partial x} A_{11}\frac{\partial u}{\partial y} 
  + Du = f
  \tag{3.4}
\end{equation}

So besides the settings $u=T^{(n)}$, $D = \frac{\rho c _{p}}{h}$ and $f = q _{H} + \frac{\rho c_p}{h} T^{(n-1)}$ as we have used before (see Equation (2.10)) we need to set

\begin{equation}\label{eqn: kappa general}
  A_{00}=A_{11}=\kappa; A_{01}=A_{10}=0 .
  \tag{3.5}
\end{equation}

The fundamental difference to the 1D case is that $A_{11}$ is not set to zero but $\kappa$, which brings in the second dimension. It is important to note that the coefficients of the PDE may depend on their location in the domain which does not influence the usage of the PDE form. This is very convenient as we can introduce spatial dependence to the PDE coefficients without modification to the way we call the PDE solver. 

A very convenient way to define the matrix $A$ in Equation (3.5) can be carried out using the [Kronecker $\delta$ symbol](http://en.wikipedia.org/wiki/Kronecker_delta). The *escript* function `kronecker` returns this matrix:

\begin{equation}
  \verb|kronecker(model)| = \left[ 
  \begin{array}{cc}
   1 & 0 \\
   0 & 1 \\
  \end{array}
  \right] .
  \tag{3.6}
\end{equation}

As the argument `model` represents a two dimensional domain the matrix is returned as a $2 \times 2$ matrix
(in the case of a three-dimensional domain a $3 \times 3$ matrix is returned). The call 

```python
mypde.setValue(A=kappa*kronecker(model),D=rhocp/h)
```

sets the PDE coefficients according to Equation (3.5). 

We need to check the boundary conditions before we turn to the question: how do we set $\kappa$. As pointed out Equation (2.4) makes certain assumptions on the boundary conditions. In our case these assumptions translate to

\begin{equation}
  -n \cdot \kappa \nabla T^{(n)} = 
  -n_{0} \cdot \kappa \frac{\partial T^{(n)}}{\partial x} -
  n_{1} \cdot  \kappa \frac{\partial T^{(n)}}{\partial y} = 0 ,
  \tag{3.7}
\end{equation}

which sets the normal component of the heat flux $- \kappa \cdot (\frac{\partial T^{(n)}}{\partial x}, \frac{\partial T^{(n)}}{\partial y})$ to zero. This means that the region is insulated which is what we want. 
On the left and right face of the domain where we have $(n_{0},n_{1} ) = (\mp 1,0)$, which means $\frac{\partial T^{(n)}}{\partial x}=0$ and on the top and bottom on the domain  where we have  $(n_{0},n_{1} ) = (\pm 1,0)$ this is $\frac{\partial T^{(n)}}{\partial y}=0$. 

## 3.2 Setting variable PDE Coefficients

Now we need to look into the problem of how we define the material coefficients $\kappa$ and $\rho c_p$ depending on their location in the domain. We can make use of the technique used in the granite block example in
Section 2.1 to set up the initial temperature. However, the situation is more complicated here as we have to deal with a curved interface between the two sub-domains.

Prior to setting up the PDE, the interface between the two materials must be established. The distance $s\ge 0$ between two points $[x,y]$ and $[x_{0},y_{0}]$ in Cartesian coordinates is defined as:

\begin{equation}
 (x-x_{0})^{2}+(y-y_{0})^{2} = s^{2} .
 \tag{3.7}
\end{equation}

If we define the point $[x_{0},y_{0}]$ as $ic$ which denotes the centre of the semi-circle of our intrusion, then for all the points $[x,y]$ in our model we can calculate a distance to $ic$. All the points that fall within the given radius $r$ of our intrusion will have a corresponding value $s < r$ and all those belonging to the country rock will have a value $s > r$. By subtracting $r$ from both of these conditions we find $s-r < 0$ for all intrusion points and $s-r > 0$ for all country rock points. Defining these conditions within the script is quite simple and is done using the following command:

In [ ]:
x=Function(model).getX()
bound = length(x-ic)-r #where the boundary will be located

print("Done")

Notice that we are using the sample points of the `Function` function space as expected for the PDE coefficient `A` (For the experienced user: use `x=mypde.getFunctionSpace("A").getX()`).This definition of the boundary can now be used with the `whereNegative` command again to help define the material constants and temperatures in our domain.  

In [ ]:
## Intrusion Variables
#Granite
kappai=2.2*W/m/K #watts/m.K thermal conductivity
rhoi = 2750*kg/m**3 #kg/m^{3} density of granite
cpi = 790.*J/(kg*K) #j/Kg.K thermal capacity
rhocpi = rhoi*cpi   #DENSITY * SPECIFIC HEAT
#Country Rock Variables - Sandstone
kappac = 1.9*W/m/K #watts/m.K thermal conductivity
rhoc = 2000*kg/m**3 #kg/m^{3} density
cpc = 920.*J/(kg*K) #j/kg.k specific heat
rhocpc = rhoc*cpc #DENSITY * SPECIFIC HEAT

kappa = kappai * whereNegative(bound) + kappac * (1-whereNegative(bound))
rhocp = rhocpi*whereNegative(bound)+rhocpc*(1-whereNegative(bound))

#Script/Iteration Related
t=0 * day #our start time, usually zero
tend=200.* yr #the time we want to end the simulation
outputs = 200 # number of time steps required.
h=(tend-t)/outputs #size of time step
#user warning
print("Expected Number of Output Files is: ", outputs)
print("Step size is: ", h/day, "days")

#create the PDE
from esys.escript.linearPDEs import LinearPDE
mypde=LinearPDE(model) #assigns a domain to our PDE
mypde.setSymmetryOn() #set the fast solver on for symmetry
#define our PDE coeffs
mypde.setValue(A=kappa*kronecker(model),D=rhocp/h)

print("Done")

Our PDE has now been properly established. The initial conditions for temperature are set out in a similar manner:

In [ ]:
#defining the initial temperatures.
Ti=2273.*Celsius # Kelvin -the starting temperature of our RHS Block
Tc = 473*Celsius # Kelvin #the starting temperature of our country rock
x=Solution(model).getX()
bound = length(x-ic)-r
T= Ti*whereNegative(bound)+Tc*(1-whereNegative(bound))

print("Done")

where `Ti` and `Tc` are the initial temperature in the regions of the granite and surrounding sandstone, respectively. It is important to notice that we reset `x` and `bound` to refer to the appropriate sample points of a PDE solution (For the experienced user: use `x=mypde.getFunctionSpace("r").getX()`).

## 3.3 Contouring *escript* Data using `matplotlib`.

To complete our transition from a 1D to a 2D model we also need to modify the plotting procedure. As before we use `matplotlib` to do the plotting in this case a contour plot. For 2D contour plots `matplotlib` expects that the data are regularly gridded. We have no control over the location and ordering of the sample points used to represent the solution. Therefore it is necessary to interpolate our solution onto a regular grid.

In Section 2.1.10 we have already learned how to extract the $x$ coordinates of sample points as `numpy` array to hand the values to `matplotlib`. This can easily be extended to extract both the $x$ and the $y$ coordinates:

In [ ]:
import numpy as np
def toXYTuple(coords):
    coords = np.array(coords.toListOfTuples()) #convert to Tuple
    coordX = coords[:,0] #X components.
    coordY = coords[:,1] #Y components.
    return coordX,coordY

print("Done")

For convenience we have put this function into `cblib.py` so we can use this function in other scripts more easily. 

We now generate a regular $100 \times 100$ grid over the domain ($mx$ and $my$ are the dimensions in the $x$ and $y$ directions) which is done using the `numpy` function `linspace`.

In [ ]:
from cblib import toXYTuple
# get sample points for temperature as for contouring
coordX, coordY = toXYTuple(T.getFunctionSpace().getX())
# create regular grid
xi = np.linspace(0.0,mx,75)
yi = np.linspace(0.0,my,75)

print("Done")

The values `[xi[k], yi[l]]` are the grid points.

The remainder of our contouring commands resides within a `while` loop so that a new contour is generated for each time step. For each time step the solution must be re-gridded for `matplotlib` using the SciPy `griddata` function. This function interprets irregularly located values `tempT` at locations defined by `coordX` and `coordY` as values at the new coordinates of a rectangular grid defined by `xi` and `yi`. The output is `zi`. It is now possible to use the `contourf` function which generates colour filled contours. The colour gradient of our plots is set via the command `pl.matplotlib.pyplot.autumn()`, other colours are listed on the `matplotlib` [web page](http://matplotlib.sourceforge.net/api/). Our results are then contoured, visually adjusted using the `matplotlib` functions and then saved to a file. `pl.clf()` clears the figure in readiness for the next time iteration.

In [ ]:
import os #This package is necessary to handle saving our data.
import matplotlib
import pylab as pl #Plotting package.
from scipy import interpolate
from cblib import HAVE_NATGRID
 
#the folder to put our outputs in, leave blank "" for script path 
save_path= os.path.join("data","example03")
mkDir(save_path)

print("Simulation started, this will take a while...")
i=0 #loop counter
while t<=tend:
    i+=1 #counter
    t+=h #current time
    print("  iteration step %d, time=%s/%s"%(i,t,tend))
    mypde.setValue(Y=qH+T*rhocp/h)
    T=mypde.getSolution()
    tempT = T.toListOfTuples()
    # grid the data.
    #zi = pl.matplotlib.mlab.griddata(coordX,coordY,tempT,xi,yi, interp='linear')
    zi = interpolate.griddata((coordX,coordY),tempT,(xi[None,:],yi[:,None]), method='linear')
    # contour the gridded data, plotting dots at the 
    # randomly spaced data points.
    pl.matplotlib.pyplot.autumn()
    pl.contourf(xi,yi,zi,10)
    CS = pl.contour(xi,yi,zi,5,linewidths=0.5,colors='k')
    pl.clabel(CS, inline=1, fontsize=8)
    pl.axis([0,600,0,600])
    pl.title("Heat diffusion from an intrusion.")
    pl.xlabel("Horizontal Displacement (m)")
    pl.ylabel("Depth (m)")
    pl.savefig(os.path.join(save_path,"temp%03d.png"%i))
    pl.clf()            
    #print("time step %s at t=%e days completed."%(i,t/day))
print("Simulation completed, please check saved figures in your output folder.")

The function `pl.contour` is used to add labelled contour lines to the plot. The results for selected time steps are shown in Figure 3.2, to which you can compare your results.

<br>
 <figure>
  <img src="figures/temp001.png" width="500">
 </figure>

 <figure>
  <img src="figures/temp020.png" width="500">
 </figure>

 <figure>
  <img src="figures/temp200.png" width="500">
  <figcaption>
    <center>
      Figure 3.2: Example 3a: 2D model: Total temperature ($T$) distribution at time steps
$1$, $20$ and $200$.Contour lines show temperature. 
    </center>
  </figcaption>
 </figure>
<br>

## 3.4 Advanced Visualisation using VTK

**The scripts referenced in this section are; example03b.py**

An alternative approach to `matplotlib` for visualisation is the usage of a package which is based on the [Visualization Toolkit (VTK) library](http://www.vtk.org/). There is a variety of packages available. Here we use the package [*mayavi2*](http://code.enthought.com/projects/mayavi/) as an example. 

*mayavi2* is an interactive, GUI driven tool which is really designed to visualise large three dimensional data sets where `matplotlib` is not suitable. But it is also very useful when it comes to two dimensional problems.  The decision of which tool is the best can be subjective and users should determine which package they require and are most comfortable with. The main difference between using *mayavi2* (or other VTK based tools) and `matplotlib` is that the actual visualisation is detached from the calculation by writing the results to external files and importing them into *mayavi2*. In 3D the best camera position for rendering a scene is not obvious before the results are available. Therefore the user may need to try different settings before the
best is found. Moreover, in many cases a 3D interactive visualisation is the only way to really understand the results (e.g. using stereographic projection).

To write the temperatures at each time step to data files in the VTK file format one needs to import `saveVTK` from the `esys.weipa` module and call it:

In [ ]:
from esys.weipa import saveVTK

#reset relevant variables to initial conditions
x=Solution(model).getX()
bound = length(x-ic)-r
T= Ti*whereNegative(bound)+Tc*(1-whereNegative(bound))
t=0 * day
i=0

print("Simulation started, this will take a while...")
while t<=tend:
    i+=1 #counter
    t+=h #current time
    print("  iteration step %d, time=%s/%s"%(i,t,tend))
    mypde.setValue(Y=qH+T*rhocp/h)
    T=mypde.getSolution()
    saveVTK(os.path.join(save_path,"data.%03d.vtu"%i), T=T)
    
print("Simulation completed, please check saved vtu files in your output folder.")    

The data files, e.g. `data.001.vtu`, contain all necessary information to visualise the temperature and can be directly processed by `mayavi2`. Note that there is no re-gridding required. The file extension `.vtu` is automatically added if not supplied to `saveVTK`. 

After you run the script you will find the result files `data.*.vtu` in the result directory `data/example03`.
Run the command

`>> mayavi2 -d data.001.vtu -m Surface &`

from the result directory. `mayavi2` will start up a window similar to Figure (3.3). The right hand side shows the temperature at the first time step. To show the results at other time steps you can click at the item `VTK XML file (data.001.vtu) (timeseries)` at the top left hand side. This will bring up a new window similar to the window shown in Figure (3.4). By clicking at the arrows in the top right corner you can move forwards and backwards in time. You will also notice the text **T** next to the item `Point scalars name`. The name **T** corresponds to the keyword argument name `T` we have used in the `saveVTK` call. In this menu item you can select other results you may have written to the output file for visualisation.

<br>
 <figure>
  <img src="figures/ScreeshotMayavi2n1.png" width="500">
  <figcaption>
    <center>
      Figure 3.3: Example 3a: Example 3b: Mayavi2 start up window. 
    </center>
  </figcaption>
 </figure>
<br>

<br>
 <figure>
  <img src="figures/ScreeshotMayavi2n2.png" width="500">
  <figcaption>
    <center>
      Figure 3.4: Example 3b: Mayavi2 data control window.
    </center>
  </figcaption>
 </figure>
<br>

**For the advanced user**: Using the `matplotlib` to visualise spatially distributed data is not MPI compatible. However, the `saveVTK` function can be used with MPI. In fact, the function collects the values of the sample
points spread across processor ranks into a single file. For more details on writing scripts for parallel computing please consult the *user's guide*.